Manon LEMAIRE - 12/2022

# **FINAL PROJECT MACHINE LEARNING**

## Text classification: goemotions 
--> Guess the sentiment(s) of the given text passage, among 28 possible labels

# **Importing libraries**

In [3]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.stem import WordNetLemmatizer
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

You should consider upgrading via the 'C:\Users\Manon Lemaire\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


# **Downloading Data**

In [4]:
train = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
test = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
evaluate = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

In [5]:
train = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv")
test = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv")
evaluate = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv")

# **Dropping unnecessary columns**

In [6]:
train = train.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)
test =test.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)
evaluate =evaluate.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)

In [7]:
train.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# **Preprocessing**


In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to C:\Users\Manon
[nltk_data]     Lemaire\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Manon
[nltk_data]     Lemaire\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Manon
[nltk_data]     Lemaire\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Manon
[nltk_data]     Lemaire\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Manon
[nltk_data]     Lemaire\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

def preprocess(sentence):
  sentence = nltk.tokenize.word_tokenize(sentence) # tokenizing
  sentence = [i.lower() for i in sentence if i not in string.punctuation] # removing punctuation
  sentence = [i for i in sentence if not i.isdigit()] # removing numbers
  sentence = [i for i in sentence if i not in stopwords] # removing stopwords
  sentence = (lemmatizer.lemmatize(w) for w in sentence) # lemmatizing 
  sentence_clean = ' '.join(w for w in sentence if w.lower() in words or not w.isalpha()) # joining the tokens

  return sentence_clean

In [10]:
before = train['text'][:3]
after = train['text'][:3].apply(preprocess)
print(f'Before: {before.values}\n\nAfter: {after} ')

Before: ['That game hurt.'
 ' >sexuality shouldn’t be a grouping category It makes you different from othet ppl so imo it fits the definition of "grouping" '
 "You do right, if you don't care then fuck 'em!"]

After: 0                                            game hurt
1    sexuality ’ grouping category make different f...
2                                   right n't care 'em
Name: text, dtype: object 


In [11]:
tqdm.pandas() # adding a loading bar on the apply process
train['text'] = train.text.progress_apply(preprocess)
test['text'] = test.text.progress_apply(preprocess)

  0%|          | 0/70000 [00:00<?, ?it/s]

  0%|          | 0/70000 [00:00<?, ?it/s]

In [12]:
labels = train.drop(['text'], axis = 1).values
labels_test = test.drop(['text'], axis = 1).values
print(labels.shape, labels_test.shape)

(70000, 28) (70000, 28)


# **Classifier pipeline**

In [13]:
pipe = Pipeline(
    [
        ('vectorizer', TfidfVectorizer()), # word vectorizer
        ('classifier', MultiOutputClassifier(XGBClassifier())) # support vector machine
    ]
)

In [14]:
pipe.fit(train.text, labels)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               feature_types=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...)))])

In [15]:
print(f'Accuracy: {accuracy_score(labels_test, pipe.predict(test.text)):.3f}')

Accuracy: 0.093


In [16]:
print(classification_report(labels_test, pipe.predict(test.text), target_names = train.drop(['text'], axis = 1).columns))

                precision    recall  f1-score   support

    admiration       0.66      0.27      0.38      5773
     amusement       0.60      0.09      0.16      3044
         anger       0.47      0.03      0.05      2762
     annoyance       0.53      0.02      0.03      4557
      approval       0.59      0.02      0.05      5785
        caring       0.42      0.04      0.07      1985
     confusion       0.58      0.04      0.07      2388
     curiosity       0.73      0.05      0.09      3190
        desire       0.41      0.06      0.10      1245
disappointment       0.42      0.02      0.03      2894
   disapproval       0.36      0.01      0.01      3798
       disgust       0.58      0.09      0.15      1753
 embarrassment       0.52      0.09      0.15       856
    excitement       0.57      0.06      0.11      1879
          fear       0.54      0.15      0.23      1067
     gratitude       0.88      0.72      0.80      3841
         grief       0.43      0.06      0.10  

C:\Users\Manon Lemaire\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Manon Lemaire\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
